# Basic Example

In [1]:
import os
import sys
srcpath = os.path.realpath('../SourceFiles')
sys.path.append(srcpath)
import pyte_visa_utils as pyte

In [2]:
%matplotlib notebook

import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from IPython.core.display import display, HTML
from scipy.signal import chirp
import matplotlib.pyplot as plt

In [3]:
# Connect to instrument

# Please choose appropriate address:
inst_addr = 'TCPIP::172.16.10.2::5025::SOCKET'

inst = pyte.open_session(inst_addr)
resp = inst.query("*IDN?")
print(resp)

Tabor Electronics,P2584M,000000220725,1.115.0


In [4]:
# Some initializations

paranoia_level = 1

cmd = "*CLS"
pyte.send_cmd(inst, cmd, paranoia_level)

cmd = "*RST"
#pyte.send_cmd(inst, cmd, paranoia_level)

cmd = ":INST:CHAN 1"
pyte.send_cmd(inst, cmd, paranoia_level)

cmd = ":FREQ:RAST 1e9"
pyte.send_cmd(inst, cmd, paranoia_level)

#cmd = ":INIT:CONT ON"
#pyte.send_cmd(inst, cmd, paranoia_level)

cmd = ":TRAC:DEL:ALL"
pyte.send_cmd(inst, cmd, paranoia_level)

In [8]:
# Get the DAC mode (8 bits or 16 bits)
resp = inst.query(':SYST:INF:DAC?')
dac_mode = resp
print('DAC mode {0} '.format(dac_mode))

if dac_mode == 'M0':
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Get number of channels
resp = inst.query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

DAC mode M0 
Number of channels: 4


In [9]:
# Build 12 waveforms

seglen = 81920
num_cycles = (1, 2, 4, 8)

waves = [ None for _ in range(12)]


for ii in range(4):
    ncycles = num_cycles[ii]
    cyclelen = seglen / ncycles
    
    x = np.linspace(
        start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
    y = (np.sin(x) + 1.0) * half_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    if dac_mode == 'M0':
        waves[ii] = y.astype(np.uint16)
    else:
        waves[ii] = y.astype(np.uint8)
    
for ii in range(4):
    ncycles = num_cycles[ii]
    cyclelen = seglen / ncycles
    
    x = np.linspace(
        start=0, stop=2 * np.pi * ncycles, num=seglen, endpoint=False)
    y = np.sin(x)
    y = np.arcsin(y)* 2 * half_dac / np.pi + half_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    if dac_mode == 'M0':
        waves[ii + 4] = y.astype(np.uint16)
    else:
        waves[ii + 4] = y.astype(np.uint8)
    
for ii in range(4):
    ncycles = num_cycles[ii]
    cyclelen = seglen / ncycles
    
    x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
    y = np.fmod(x, cyclelen)
    y = (y <= cyclelen / 2) * max_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    if dac_mode == 'M0':
        waves[ii + 8] = y.astype(np.uint16)
    else:
        waves[ii + 8] = y.astype(np.uint8)
    

In [10]:
# Download segments

# data-type code: 'B' = uint8, 'H' = uint16
data_type = 'H' if dac_mode == 'M0' else 'B'

for ii in range(num_channels):
    ichan = ii
    channb = ichan + 1
    segnum = ichan % 2 + 1
    wav = waves[ichan]
    print('Download wave to segment {0} of channel {1}'.format(segnum, channb))
    
    cmd = ':INST:CHAN {0}'.format(channb)
    pyte.send_cmd(inst, cmd, paranoia_level)
    
    cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
    pyte.send_cmd(inst, cmd, paranoia_level)
    
    cmd = ':TRAC:SEL {0}'.format(segnum)
    pyte.send_cmd(inst, cmd, paranoia_level)
    
    inst.timeout = 30000
    
    inst.write_binary_values(
        '*OPC?; :TRAC:DATA', wav, datatype=data_type, is_big_endian=False) # B = uint8, H = uint16
    
    resp = inst.read();
    inst.timeout = 10000;
    
    resp = inst.query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{0}" after writing binary values'.format(resp))

    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
    pyte.send_cmd(inst, cmd, paranoia_level)

    cmd = ':OUTP ON'
    pyte.send_cmd(inst, cmd, paranoia_level)
    
    resp = inst.query(':SYST:ERR?')
    print(resp)

print()
#cmd = ":FREQ:RAST 2.5e9"
#pyte.send_cmd(inst, cmd, paranoia_level)

resp = inst.query(':SYST:ERR?')
print(resp)


Download wave to segment 1 of channel 1
0, no error
Download wave to segment 2 of channel 2
0, no error
Download wave to segment 1 of channel 3
0, no error
Download wave to segment 2 of channel 4
0, no error

0, no error


In [11]:
paranoia_level = 0
cmd = ':INST:CHAN 1'
pyte.send_cmd(inst, cmd, paranoia_level)

In [12]:
cmd = ":FREQ:RAST 2e9"
pyte.send_cmd(inst, cmd, paranoia_level)

In [13]:
import tempfile
import webbrowser

channb = 1
cmd = ':INST:CHAN {0}; :SYST:INF:REG?'.format(channb)
html_str = inst.query(':INST:CHAN 1; :SYST:INF:REG?')
with tempfile.NamedTemporaryFile('w', delete=False, suffix='.html') as f:
    url = 'file://' + f.name
    f.write(html_str)
webbrowser.open(url)

True

In [ ]:
#html_widget.value = html_str

#html_widget = widgets.HTML(
#    value="",
#    placeholder='Registers',
#    description='Registers',
#)

#channel_widget = widgets.IntText(
#    value=1,
#    description='Channel',
#    disabled=False
#)

#button_widget = widgets.Button(description="Read Registers")

#def on_button_clicked(b):
#    global channel_widget
#    global html_widget
#    global inst
#    
#    chan = channel_widget.value
#    cmd = ':INST:CHAN {0}; :SYST:INF:REG?'.format(chan)
#    html_str = inst.query(cmd)
#    html_widget.value = html_str

    
#button_widget.on_click(on_button_clicked)
#html_str = inst.query(':INST:CHAN 1; :SYST:INF:REG?')
#html_widget.value = html_str
#display(channel_widget, button_widget, html_widget)